In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import os
import holidays
import datetime as dt
import networkx as nx
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import glob
os.getcwd()
pd.set_option('display.max_columns', 70)

In [2]:
#For commercial stop
### PLEASE CHANGE THE FOLDER PATH
all_comstop = pd.read_parquet("~/Downloads/tjj_allptcars")
all_comstop = all_comstop[(all_comstop["REF_REPERAGE_NO"] > 0) & (all_comstop["COMMERCIAL_STOP_NATURE"] != 0)] #Filter for all comercial stops
hours_range = [5, 6, 7, 8, 9, 10, 15, 16, 17, 18, 19, 20]
all_comstop = all_comstop[all_comstop['THR_ARR'].dt.hour.isin(hours_range)]
all_comstop['TRAIN_DEPARTURE_DATE'] = pd.to_datetime(all_comstop['TRAIN_DEPARTURE_DATE'], format='%Y-%m-%d')
all_comstop

,REF_TOD_TRAIN_ID,TRAIN_PREFIX,TRAIN_NUMBER,TRAIN_DEPARTURE_DATE,REF_PTCAR_NO,PTCAR_ORD_NO,THR_ARR,THR_DEP,HR_ARR,HR_DEP,EH_ARR,EH_DEP,REF_REPERAGE_NO,DEPARTURE_AT_DT_ORIGIN,ARRIVAL_AT_DT_ORIGIN,COMMERCIAL_STOP_NATURE,DEPARTURE_REF_LINE_SECTION_NO,LINE,IDENTIFICATOR
0,31520427,None,2067,2022-11-23,6,124,2022-11-23 18:54:00,1900-01-01 00:00:00,2022-11-23 18:53:24,1900-01-01 00:00:00,-35,0,1,1900-01-01 00:00:00,2022-11-23 18:54:00,3,0,NaN,None
19,30391236,None,2539,2022-05-30,227,134,2022-05-30 19:24:00,2022-05-30 19:26:00,2022-05-30 19:29:10,2022-05-30 19:30:06,310,246,1,2022-05-30 19:26:00,2022-05-30 19:24:00,2,1656,161.0,161A
27,32494329,None,4189,2023-05-08,895,103,2023-05-08 18:12:00,2023-05-08 18:15:00,2023-05-08 18:12:10,2023-05-08 18:19:47,10,287,1,2023-05-08 18:15:00,2023-05-08 18:12:00,2,1485,130.0,130
32,30529561,None,5964,2022-06-21,502,106,2022-06-21 15:51:00,2022-06-21 15:53:00,2022-06-21 15:51:24,2022-06-21 15:53:00,24,0,1,2022-06-21 15:53:00,2022-06-21 15:51:00,2,1282,165.0,165
36,30288983,None,2138,2022-05-13,1106,101,2022-05-13 15:24:00,2022-05-13 15:36:00,2022-05-13 15:32:18,2022-05-13 15:41:24,498,324,1,2022-05-13 15:24:00,2022-05-13 15:24:00,2,532,162.0,162
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36044904,32465860,None,7883,2023-05-03,848,105,2023-05-03 07:27:00,1900-01-01 00:00:00,2023-05-03 07:25:25,1900-01-01 00:00:00,-94,0,1,1900-01-01 00:00:00,2023-05-03 07:27:00,3,0,NaN,None
36044905,32467539,None,5990,2023-05-03,723,116,2023-05-03 19:59:00,1900-01-01 00:00:00,2023-05-03 19:57:57,1900-01-01 00:00:00,-62,0,1,1900-01-01 00:00:00,2023-05-03 19:59:00,3,0,NaN,None
36044907,32467053,None,6066,2023-05-03,723,121,2023-05-03 18:47:00,1900-01-01 00:00:00,2023-05-03 18:46:27,1900-01-01 00:00:00,-32,0,1,1900-01-01 00:00:00,2023-05-03 18:47:00,3,0,NaN,None
36044910,32420846,None,7868,2023-04-25,848,114,2023-04-25 07:53:00,1900-01-01 00:00:00,2023-04-25 07:56:48,1900-01-01 00:00:00,228,0,1,1900-01-01 00:00:00,2023-04-25 07:53:00,3,0,NaN,None


In [3]:
PTCAR = pd.read_csv("./data/PTCAR.csv",sep=";")
all_comstop_ptcar = all_comstop.merge(PTCAR,on='REF_PTCAR_NO',how = 'left')
all_comstop_ptcar = all_comstop_ptcar[all_comstop_ptcar['LONG_NAME_NL'].notna()]
all_comstop_ptcar = all_comstop_ptcar[all_comstop_ptcar['THR_ARR'] != '1900-01-01 00:00:00'].reset_index()
#Any trains arrive within HR_ARR_3min and HR_ARR are behind the the target train
all_comstop_ptcar['THR_ARR_3min'] = all_comstop_ptcar['THR_ARR'] + timedelta(minutes=3)
all_comstop_ptcar = all_comstop_ptcar[['REF_TOD_TRAIN_ID','THR_ARR','THR_ARR_3min','LONG_NAME_NL','TRAIN_DEPARTURE_DATE']]
all_comstop_ptcar = all_comstop_ptcar.reset_index()
all_comstop_ptcar = all_comstop_ptcar.drop(columns='index')
all_comstop_ptcar

,REF_TOD_TRAIN_ID,THR_ARR,THR_ARR_3min,LONG_NAME_NL,TRAIN_DEPARTURE_DATE
0,31520427,2022-11-23 18:54:00,2022-11-23 18:57:00,AALST,2022-11-23
1,30391236,2022-05-30 19:24:00,2022-05-30 19:27:00,BRUSSEL-SCHUMAN,2022-05-30
2,32494329,2023-05-08 18:12:00,2023-05-08 18:15:00,NAMUR,2023-05-08
3,30529561,2022-06-21 15:51:00,2022-06-21 15:54:00,HALANZY,2022-06-21
4,30288983,2022-05-13 15:24:00,2022-05-13 15:27:00,STERPENICH-FRONTIERE,2022-05-13
...,...,...,...,...,...
8341329,32465860,2023-05-03 07:27:00,2023-05-03 07:30:00,MONS,2023-05-03
8341330,32467539,2023-05-03 19:59:00,2023-05-03 20:02:00,LIBRAMONT,2023-05-03
8341331,32467053,2023-05-03 18:47:00,2023-05-03 18:50:00,LIBRAMONT,2023-05-03
8341332,32420846,2023-04-25 07:53:00,2023-04-25 07:56:00,MONS,2023-04-25


In [4]:
folder_path = '/Users/miek/Downloads/behind_comstops'
all_comstop_ptcar = all_comstop_ptcar.sort_values(by = 'REF_TOD_TRAIN_ID')
#unique_dates = test_df['TRAIN_DEPARTURE_DATE'].unique()
grouped_data = []

grouped = all_comstop_ptcar.groupby(['LONG_NAME_NL'])
total_groups = len(grouped)
current_group = 0

for name, group in grouped:
    current_group += 1
    print(f"Processing group: {current_group}/{total_groups} - {name}")
    
    ref_ids = group['REF_TOD_TRAIN_ID'].values
    mask = (ref_ids[:, np.newaxis] != ref_ids) & \
    (group['THR_ARR'].values[:, np.newaxis] >= group['THR_ARR'].values) & \
    (group['THR_ARR'].values[:, np.newaxis] <= group['THR_ARR_3min'].values)
    ohe = pd.get_dummies(group['REF_TOD_TRAIN_ID'])
    np.fill_diagonal(ohe.values, 0)
    group_ohe = pd.concat([group, ohe], axis=1)
    group_ohe.iloc[:, 5:] = np.where(mask, 1, group_ohe.iloc[:, 5:])
    group_ohe = group_ohe.drop(columns = ['THR_ARR','THR_ARR_3min','LONG_NAME_NL','TRAIN_DEPARTURE_DATE'])
    group_ohe = group_ohe.set_index(['REF_TOD_TRAIN_ID'])
    group_ohe = group_ohe.T
    group_ohe['num_train_behind'] = group_ohe.sum(axis='columns')
    group_ohe['LONG_NAME_NL'] = name[0]
    group_ohe = group_ohe[['num_train_behind','LONG_NAME_NL']]
    name_with_underscores = name[0].replace(" ", "-").replace('/', '').replace(':', '')
    filename = f"{name_with_underscores}.csv"  # Using 'LONG_NAME_NL' and day in the filename
    #Save the group dataframe as a CSV file in the specified folder
    group_ohe.to_csv(os.path.join(folder_path, filename), index=True)



Processing group: 1/690 - ('AALST',)
Processing group: 2/690 - ('AALST-KERREBROEK',)
Processing group: 3/690 - ('AALTER',)
Processing group: 4/690 - ('AARSCHOT',)
Processing group: 5/690 - ('AARSELE',)
Processing group: 6/690 - ('ACREN',)
Processing group: 7/690 - ('AISEAU',)
Processing group: 8/690 - ('ALKEN',)
Processing group: 9/690 - ('AMAY',)
Processing group: 10/690 - ('AMPSIN',)
Processing group: 11/690 - ('ANDENNE',)
Processing group: 12/690 - ('ANDERLECHT',)
Processing group: 13/690 - ('ANGLEUR',)
Processing group: 14/690 - ('ANS',)
Processing group: 15/690 - ('ANS-PLANS',)
Processing group: 16/690 - ('ANSEREMME',)
Processing group: 17/690 - ('ANTOING',)
Processing group: 18/690 - ('ANTWERPEN-BERCHEM',)


: 

: 

In [13]:
name[0]

'AALST'

In [11]:
type(name)

tuple

In [9]:
group_ohe['LONG_NAME_NL'] = name

ValueError: Length of values (1) does not match length of index (24524)